<a href="https://colab.research.google.com/github/sanjaynagi/Ano-expressIR/blob/main/workflow/notebooks/expression-candidates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

def load_candidates(analysis, name='median', func=np.nanmedian, query_annotation=None, query_fc=None):
    fc_data = pd.read_csv(f"https://raw.githubusercontent.com/sanjaynagi/ano-expressir/main/results/fcs.{analysis}.tsv", sep="\t")
    fc_data = fc_data.set_index(['GeneID', 'GeneName', 'GeneDescription'])

    if query_annotation is not None:
      gene_annot_df = load_annotations()
      gene_ids = gene_ids_from_annotation(gene_annot_df=gene_annot_df, annotation=query_annotation)
      fc_data = fc_data.query("GeneID in @gene_ids")
      assert not fc_data.empty, "No genes were found for the selection. It is possible these genes were removed by the ortholog finding process"
    
    fc_ranked = fc_data.apply(func, axis=1).to_frame().rename(columns={0:f'{name} log2 Fold Change'}).copy()
    fc_ranked = fc_ranked.sort_values(f'{name} log2 Fold Change', ascending=False)
    fc_ranked = fc_ranked.reset_index()
    fc_ranked.loc[:, f'{name} Fold Change'] = np.round(2**fc_ranked.loc[:, f'{name} log2 Fold Change'], 2)

    if query_fc is not None:
      fc_ranked = fc_ranked.query(f'`{name} Fold Change` > {query_fc}')

    return(fc_ranked)
    
def gene_ids_from_annotation(gene_annot_df, annotation):
    if isinstance(annotation, list):
        gene_list = np.array([])
        if annotation[0].startswith("GO"):
            for go in annotation:
                ids = gene_annot_df.query(f"GO_terms.str.contains('{go}', na=False)", engine='python')['gene_id'].to_numpy()
                gene_list = np.hstack([gene_list, ids])
            return(np.unique(gene_list))
        else:
            for dom in annotation:
                ids = gene_annot_df.query("domain == @annotation")['gene_id'].to_numpy()
                gene_list = np.hstack([gene_list, ids])
            return(np.unique(gene_list))
    else:
        if annotation.startswith("GO"): 
            return(gene_annot_df.query(f"GO_terms.str.contains('{annotation}', na=False)", engine='python')['gene_id'].to_numpy())
        else:
            return(gene_annot_df.query("domain == @annotation")['gene_id'].to_numpy())

def load_annotations():
    pfam_df = pd.read_csv("https://github.com/sanjaynagi/ano-expressir/blob/main/resources/Anogam_long.pep_Pfamscan.seqs.gz?raw=true", sep="\s+", header=None, compression='gzip')
    go_df = pd.read_csv("https://github.com/sanjaynagi/ano-expressir/blob/main/resources/Anogam_long.pep_eggnog_diamond.emapper.annotations.GO.gz?raw=true", sep="\t", header=None, compression='gzip')
    pfam_df.columns = ["transcript", "pstart", "pend", "pfamid", "domain", "domseq"]
    go_df.columns = ['transcript', 'GO_terms']

    gene_annot_df = pfam_df.merge(go_df)
    gene_annot_df.loc[:, 'gene_id'] = gene_annot_df.loc[:, 'transcript'].str.replace("Anogam_", "").str.replace("-R[A-Z]", "", regex=True)
    return(gene_annot_df)

### **Ano-expressIR** - Ranking genes by median and mean expression

In this notebook, you can use the function `load_candidates()` to rank genes by their mean or median fold change (or any custom metric), to identify candidate genes for an involvement in insecticide resistance.

In [ ]:
fc_median = load_candidates(analysis="gamb_colu_arab_fun", name="median", func=np.nanmedian)
fc_median.head(200)

,GeneID,GeneName,GeneDescription,median log2 Fold Change,median Fold Change
0,AGAP000047,CPR130,cuticular protein RR-2 family 130 [Source:VB C...,3.090,8.51
1,AGAP001684,NaN,Alkaline phosphatase [Source:UniProtKB/TrEMBL;...,2.770,6.82
2,AGAP002894,CYP6Z4,cytochrome P450 [Source:VB Community Annotation],2.730,6.63
3,AGAP028557,NaN,NaN,2.620,6.15
4,AGAP028402,NaN,NaN,2.590,6.02
5,AGAP008447,CPLCG4,cuticular protein CPLCG family (CPLCG4) [Sourc...,2.570,5.94
6,AGAP006417,NaN,venom allergen [Source:VB Community Annotation],2.510,5.70
7,AGAP008448,NaN,NaN,2.510,5.70
8,AGAP002867,CYP6P4,cytochrome P450 [Source:VB Community Annotation],2.510,5.70
9,AGAP006149,CPLCX3,cuticular protein unclassified [Source:VB Comm...,2.460,5.50


### **Rank by mean fold change**

In [ ]:
fc_means = load_candidates(analysis="gamb_colu_arab_fun", name='mean', func=np.nanmean)
fc_means.head(200)

,GeneID,GeneName,GeneDescription,mean log2 Fold Change,mean Fold Change
0,AGAP002865,CYP6P3,cytochrome P450 [Source:VB Community Annotation],2.764643,6.80
1,AGAP028402,NaN,NaN,2.691923,6.46
2,AGAP002867,CYP6P4,cytochrome P450 [Source:VB Community Annotation],2.603929,6.08
3,AGAP010109,CPR150,cuticular protein 150 [Source:VB Community Ann...,2.305000,4.94
4,AGAP001684,NaN,Alkaline phosphatase [Source:UniProtKB/TrEMBL;...,2.145714,4.43
5,AGAP007445,NaN,NaN,2.099643,4.29
6,AGAP002743,NaN,NaN,2.085357,4.24
7,AGAP006417,NaN,venom allergen [Source:VB Community Annotation],2.026786,4.07
8,AGAP008447,CPLCG4,cuticular protein CPLCG family (CPLCG4) [Sourc...,2.014286,4.04
9,AGAP008817,CPLCP3,cuticular protein (putative) CPLCP3 [Source:VB...,1.989231,3.97


### **Finding genes that are consistently over-expressed**

Below, you can use the function `consistent_genes()` to identify genes that are consistently expressed in the same direction. 

In [4]:
def consistent_genes(analysis, direction, n):
  import pandas as pd
  import numpy as np 
    
  fc_data = pd.read_csv(f"https://raw.githubusercontent.com/sanjaynagi/ano-expressir/main/results/fcs.{analysis}.tsv", sep="\t")
  print(f"There are {fc_data.shape[0]} genes and {fc_data.shape[1]} differential expression comparisons in {analysis}")
  fc_data = fc_data.set_index(['GeneID', 'GeneName', 'GeneDescription'])
  if direction == 'up':
    res_df = fc_data[fc_data.apply(lambda x: (x > 0).sum() >= n , axis=1)]
    if res_df.empty: 
      print(f"There are no genes expressed direction={direction} in {n} experiments")
      return
    else:
      return(res_df)
  else: 
    res_df = fc_data[fc_data.apply(lambda x: (x < 0).sum() >= n, axis=1)]
    if res_df.empty:
      print(f"There are no genes expressed {direction} in {n} experiments")
      return(res_df)

In [7]:
consistent_genes(analysis="gamb_colu_arab_fun", direction="up", n=20)

There are 8599 genes and 31 differential expression comparisons in gamb_colu_arab_fun


,,,Tiefora_v_Ngousso_log2FoldChange,Ban_v_BanS_log2FoldChange,BanRe_v_BanS_log2FoldChange,Bak_v_Kisumu_log2FoldChange,VK7_v_Kisumu_log2FoldChange,Cameroon_v_Ngousso_log2FoldChange,Chad_v_Ngousso_log2FoldChange,Niger_v_Ngousso_log2FoldChange,Nigeria_v_Ngousso_log2FoldChange,Agboville_v_Mali_log2FoldChange,...,Asendabo_v_Moz_log2FoldChange,Chewaka_v_Moz_log2FoldChange,Tolay_v_Moz_log2FoldChange,Ethiopia_v_Dongola_log2FoldChange,Gou_v_Moz_log2FoldChange,Cam_fun_v_Fang_log2FoldChange,Fumoz_v_Fang_log2FoldChange,Ghana_fun_v_Fang_log2FoldChange,Malawi_fun_v_Fang_log2FoldChange,Uganda_fun_v_Fang_log2FoldChange
GeneID,GeneName,GeneDescription,,,,,,,,,,,,,,,,,,,,,
AGAP002865,CYP6P3,cytochrome P450 [Source:VB Community Annotation],4.26,1.36,-0.76,2.03,3.90,1.26,1.87,0.41,-0.32,3.82,...,2.41,4.70,2.85,0.47,1.07,1.46,5.19,3.58,5.44,2.61
AGAP008218,CYP6Z2,cytochrome P450 [Source:VB Community Annotation],2.64,0.58,-2.17,2.55,4.33,3.46,1.54,2.07,1.50,1.35,...,1.27,1.98,0.70,-0.85,1.14,-0.20,-0.02,0.52,0.55,0.45
AGAP004382,GSTD3,glutathione S-transferase delta class 3 [Source:VB Community Annotation],2.05,1.44,0.39,-0.25,0.90,-0.39,0.67,0.18,1.22,2.09,...,1.72,1.31,1.71,1.75,1.05,0.83,1.24,1.92,1.79,1.38
AGAP008227,NaN,trehalose 6-phosphate synthase/phosphatase [Source:VB Community Annotation],-1.62,-0.82,0.37,0.33,0.53,1.32,1.67,1.92,1.62,-0.96,...,0.32,0.40,0.78,1.12,1.31,0.16,-0.07,0.06,-0.08,0.13
AGAP010675,LRIM18,leucine-rich immune protein (Coil-less) [Source:VB Community Annotation],1.15,0.52,-0.60,0.33,-0.13,0.44,0.49,0.57,0.44,0.41,...,0.05,0.03,-0.07,0.17,0.19,-0.09,-0.04,0.02,-0.01,-0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGAP009410,Or18,odorant receptor 18 [Source:VB Community Annotation],-0.45,NaN,0.88,0.22,-0.07,1.64,0.06,1.74,1.07,0.59,...,NaN,1.82,2.25,0.21,NaN,-0.20,-0.64,1.20,1.79,0.51
AGAP013339,NaN,NaN,0.51,-0.15,-1.26,0.33,1.77,0.75,-0.66,-0.42,0.66,-0.21,...,3.02,2.42,2.32,-0.55,-0.24,0.61,0.49,1.47,1.29,0.27
AGAP028402,NaN,NaN,-1.84,3.13,-1.47,NaN,2.17,1.41,1.03,2.59,NaN,2.79,...,5.32,6.56,5.17,-1.74,0.52,3.36,2.41,2.75,3.83,3.35


In [ ]:
consistent_genes("gamb_colu_arab_fun", "up", 24)

There are 8599 genes and 31 differential expression comparisons in gamb_colu_arab_fun


,,,Tiefora_v_Ngousso_log2FoldChange,Ban_v_BanS_log2FoldChange,BanRe_v_BanS_log2FoldChange,Bak_v_Kisumu_log2FoldChange,VK7_v_Kisumu_log2FoldChange,Cameroon_v_Ngousso_log2FoldChange,Chad_v_Ngousso_log2FoldChange,Niger_v_Ngousso_log2FoldChange,Nigeria_v_Ngousso_log2FoldChange,Agboville_v_Mali_log2FoldChange,...,Asendabo_v_Moz_log2FoldChange,Chewaka_v_Moz_log2FoldChange,Tolay_v_Moz_log2FoldChange,Ethiopia_v_Dongola_log2FoldChange,Gou_v_Moz_log2FoldChange,Cam_fun_v_Fang_log2FoldChange,Fumoz_v_Fang_log2FoldChange,Ghana_fun_v_Fang_log2FoldChange,Malawi_fun_v_Fang_log2FoldChange,Uganda_fun_v_Fang_log2FoldChange
GeneID,GeneName,GeneDescription,,,,,,,,,,,,,,,,,,,,,
AGAP002865,CYP6P3,cytochrome P450 [Source:VB Community Annotation],4.26,1.36,-0.76,2.03,3.90,1.26,1.87,0.41,-0.32,3.82,...,2.41,4.70,2.85,0.47,1.07,1.46,5.19,3.58,5.44,2.61
AGAP008218,CYP6Z2,cytochrome P450 [Source:VB Community Annotation],2.64,0.58,-2.17,2.55,4.33,3.46,1.54,2.07,1.50,1.35,...,1.27,1.98,0.70,-0.85,1.14,-0.20,-0.02,0.52,0.55,0.45
AGAP004382,GSTD3,glutathione S-transferase delta class 3 [Source:VB Community Annotation],2.05,1.44,0.39,-0.25,0.90,-0.39,0.67,0.18,1.22,2.09,...,1.72,1.31,1.71,1.75,1.05,0.83,1.24,1.92,1.79,1.38
AGAP004410,NaN,NaN,0.96,0.42,-0.85,0.66,0.28,0.15,0.32,0.45,0.59,0.18,...,0.57,0.46,0.59,0.04,0.02,0.20,0.17,0.11,0.29,0.06
AGAP011477,NaN,eupolytin [Source:VB Community Annotation],2.64,1.58,0.24,1.40,0.34,1.21,1.34,0.73,0.27,3.75,...,0.71,0.62,-0.07,-0.16,-0.99,0.17,0.38,1.04,0.26,0.23
AGAP002867,CYP6P4,cytochrome P450 [Source:VB Community Annotation],2.05,2.56,0.59,1.65,4.70,2.30,0.59,-0.28,1.11,3.89,...,2.51,3.12,3.14,2.03,-0.85,1.08,5.16,3.49,5.41,2.51
AGAP000476,NaN,Xaa-Pro aminopeptidase [Source:VB Community Annotation],1.23,0.45,-1.63,1.17,1.80,0.14,0.46,-0.55,-0.94,0.28,...,0.46,0.67,0.55,0.46,-0.23,0.68,0.55,0.38,0.33,0.29
AGAP013128,CYP6AA2,cytochrome P450 [Source:VB Community Annotation],0.85,1.18,0.18,0.33,0.14,0.60,0.35,0.02,0.60,2.38,...,0.31,0.80,0.22,-0.35,-0.68,0.62,0.78,0.82,0.85,-0.25
AGAP000818,CYP9K1,cytochrome P450 [Source:VB Community Annotation],0.64,1.08,-0.73,4.65,2.00,1.58,0.44,0.31,1.08,3.45,...,2.27,2.89,2.28,1.37,0.20,0.00,0.92,1.32,0.93,2.91
